In [1]:
# Library Import
import pandas as pd
import numpy as np
import math # 수학 계산을 위한 모듈

import geopandas as gpd # GeoDataFrame 을 위한 Library

from shapely.geometry import LineString, Point, Polygon, MultiPolygon

import folium # Web 지도 시각화 Library

# 시각화 Tool
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/HMFMMUEX.TTC" # 한글 꺠짐 해결을 위한 폰트 지정 
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

%matplotlib inline
np.random.seed(0)

print(gpd.__version__)

0.10.2


## 시군구 코드

In [21]:
sgg_cd = pd.read_excel('../data/sgg_cd/sgg.xlsx', dtype =str) # 전국 시군구 리스트

sgg_cd = sgg_cd.loc[(sgg_cd['행정구역코드'].str.len()==5)].reset_index().drop(columns=['index'])
sgg_cd['비자치구'] = sgg_cd['행정구역코드'].apply(lambda x: '비자치구' if x[-1] !='0' else '자치구')
sgg_cd['시군구_법정동코드'] = sgg_cd['법정동코드'].apply(lambda x: x[:5])

# 법정동코드의 경우 자치구인 경우에도 뒤에 0이 아닌 수가 나올 수 가 있음
sgg_cd['자치구명'] = sgg_cd.apply(lambda x: x['행정구역명'].split(' ')[0] if x['행정구역코드'][-1] !='0' else x['행정구역명'].split(' ')[0], axis= 1 )
sgg_cd['자치구_행정구역코드'] = sgg_cd['행정구역코드'].apply(lambda x: x[:-1]+'0' if x[-1] !='0' else x)
sgg_cd['자치구_법정동코드'] = sgg_cd['시군구_법정동코드']

sgg_cd = sgg_cd.rename(columns = {'행정구역코드':'시군구_행정구역코드'})
sgg_cd = sgg_cd[['시도', '시군구', '행정구역명', '시군구_행정구역코드', '시군구_법정동코드', '자치구_행정구역코드', '자치구_법정동코드', '비자치구', '자치구명']]

sgg_cd

,시도,시군구,행정구역명,시군구_행정구역코드,시군구_법정동코드,자치구_행정구역코드,자치구_법정동코드,비자치구,자치구명
0,서울특별시,종로구,종로구,11010,11110,11010,11110,자치구,종로구
1,서울특별시,중구,중구,11020,11140,11020,11140,자치구,중구
2,서울특별시,용산구,용산구,11030,11170,11030,11170,자치구,용산구
3,서울특별시,성동구,성동구,11040,11200,11040,11200,자치구,성동구
4,서울특별시,광진구,광진구,11050,11215,11050,11215,자치구,광진구
...,...,...,...,...,...,...,...,...,...
256,경상남도,함양군,함양군,38380,48870,38380,48870,자치구,함양군
257,경상남도,거창군,거창군,38390,48880,38390,48880,자치구,거창군
258,경상남도,합천군,합천군,38400,48890,38400,48890,자치구,합천군
259,제주특별자치도,제주시,제주시,39010,50110,39010,50110,자치구,제주시


In [23]:
autonomous_district_ad2le_dict = {}

autonomous_district_df = sgg_cd[sgg_cd['비자치구']=='자치구'].reset_index().drop(columns=['index'])

for i in autonomous_district_df.index:
    district_ad_cd = autonomous_district_df['시군구_행정구역코드'][i]
    district_le_cd = autonomous_district_df['시군구_법정동코드'][i]
    
    autonomous_district_ad2le_dict[district_ad_cd] = district_le_cd
    
autonomous_district_ad2le_dict

{'11010': '11110',
 '11020': '11140',
 '11030': '11170',
 '11040': '11200',
 '11050': '11215',
 '11060': '11230',
 '11070': '11260',
 '11080': '11290',
 '11090': '11305',
 '11100': '11320',
 '11110': '11350',
 '11120': '11380',
 '11130': '11410',
 '11140': '11440',
 '11150': '11470',
 '11160': '11500',
 '11170': '11530',
 '11180': '11545',
 '11190': '11560',
 '11200': '11590',
 '11210': '11620',
 '11220': '11650',
 '11230': '11680',
 '11240': '11710',
 '11250': '11740',
 '21010': '26110',
 '21020': '26140',
 '21030': '26170',
 '21040': '26200',
 '21050': '26230',
 '21060': '26260',
 '21070': '26290',
 '21080': '26320',
 '21090': '26350',
 '21100': '26380',
 '21110': '26410',
 '21120': '26440',
 '21130': '26470',
 '21140': '26500',
 '21150': '26530',
 '21310': '26710',
 '22010': '27110',
 '22020': '27140',
 '22030': '27170',
 '22040': '27200',
 '22050': '27230',
 '22060': '27260',
 '22070': '27290',
 '22310': '27710',
 '23010': '28110',
 '23020': '28140',
 '23040': '28185',
 '23050': '2

In [25]:
sgg_cd['자치구_법정동코드'] = sgg_cd['자치구_행정구역코드'].apply(lambda x: autonomous_district_ad2le_dict[x])
sgg_cd

,시도,시군구,행정구역명,시군구_행정구역코드,시군구_법정동코드,자치구_행정구역코드,자치구_법정동코드,비자치구,자치구명
0,서울특별시,종로구,종로구,11010,11110,11010,11110,자치구,종로구
1,서울특별시,중구,중구,11020,11140,11020,11140,자치구,중구
2,서울특별시,용산구,용산구,11030,11170,11030,11170,자치구,용산구
3,서울특별시,성동구,성동구,11040,11200,11040,11200,자치구,성동구
4,서울특별시,광진구,광진구,11050,11215,11050,11215,자치구,광진구
...,...,...,...,...,...,...,...,...,...
256,경상남도,함양군,함양군,38380,48870,38380,48870,자치구,함양군
257,경상남도,거창군,거창군,38390,48880,38390,48880,자치구,거창군
258,경상남도,합천군,합천군,38400,48890,38400,48890,자치구,합천군
259,제주특별자치도,제주시,제주시,39010,50110,39010,50110,자치구,제주시


In [27]:
sgg_cd[sgg_cd['시군구'].str.contains('수원')]

,시도,시군구,행정구역명,시군구_행정구역코드,시군구_법정동코드,자치구_행정구역코드,자치구_법정동코드,비자치구,자치구명
75,경기도,수원시,수원시,31010,41110,31010,41110,자치구,수원시
76,경기도,수원시 장안구,수원시 장안구,31011,41111,31010,41110,비자치구,수원시
77,경기도,수원시 권선구,수원시 권선구,31012,41113,31010,41110,비자치구,수원시
78,경기도,수원시 팔달구,수원시 팔달구,31013,41115,31010,41110,비자치구,수원시
79,경기도,수원시 영통구,수원시 영통구,31014,41117,31010,41110,비자치구,수원시


In [28]:
sgg_cd[sgg_cd['시군구'].str.contains('광진')]

,시도,시군구,행정구역명,시군구_행정구역코드,시군구_법정동코드,자치구_행정구역코드,자치구_법정동코드,비자치구,자치구명
4,서울특별시,광진구,광진구,11050,11215,11050,11215,자치구,광진구


## 시군구 GeoDataFrame

In [29]:
sgg_gdf = gpd.GeoDataFrame.from_file('../data/shp/sgg/SIG.shp', encoding='CP949', crs="epsg:5179").to_crs('epsg:4326')
sgg_gdf = sgg_gdf[['SIG_CD','geometry']]
sgg_gdf = sgg_gdf.rename(columns = {'SIG_CD':'시군구_법정동코드'})
sgg_gdf

,시군구_법정동코드,geometry
0,11110,"POLYGON ((127.00864 37.58047, 127.00871 37.580..."
1,11140,"POLYGON ((127.02314 37.57196, 127.02336 37.571..."
2,11170,"POLYGON ((126.96918 37.55566, 126.96917 37.554..."
3,11200,"POLYGON ((127.04341 37.57234, 127.04524 37.571..."
4,11215,"POLYGON ((127.10166 37.57240, 127.10224 37.572..."
...,...,...
245,48870,"POLYGON ((127.69596 35.75773, 127.69647 35.757..."
246,48880,"POLYGON ((127.88313 35.90586, 127.88299 35.905..."
247,48890,"POLYGON ((128.09861 35.83067, 128.09899 35.830..."
248,50110,"MULTIPOLYGON (((126.15092 33.30877, 126.15090 ..."


In [30]:
sgg_gdf = gpd.GeoDataFrame(pd.merge(sgg_cd, sgg_gdf, on='시군구_법정동코드', how='left'))
sgg_gdf

,시도,시군구,행정구역명,시군구_행정구역코드,시군구_법정동코드,자치구_행정구역코드,자치구_법정동코드,비자치구,자치구명,geometry
0,서울특별시,종로구,종로구,11010,11110,11010,11110,자치구,종로구,"POLYGON ((127.00864 37.58047, 127.00871 37.580..."
1,서울특별시,중구,중구,11020,11140,11020,11140,자치구,중구,"POLYGON ((127.02314 37.57196, 127.02336 37.571..."
2,서울특별시,용산구,용산구,11030,11170,11030,11170,자치구,용산구,"POLYGON ((126.96918 37.55566, 126.96917 37.554..."
3,서울특별시,성동구,성동구,11040,11200,11040,11200,자치구,성동구,"POLYGON ((127.04341 37.57234, 127.04524 37.571..."
4,서울특별시,광진구,광진구,11050,11215,11050,11215,자치구,광진구,"POLYGON ((127.10166 37.57240, 127.10224 37.572..."
...,...,...,...,...,...,...,...,...,...,...
256,경상남도,함양군,함양군,38380,48870,38380,48870,자치구,함양군,"POLYGON ((127.69596 35.75773, 127.69647 35.757..."
257,경상남도,거창군,거창군,38390,48880,38390,48880,자치구,거창군,"POLYGON ((127.88313 35.90586, 127.88299 35.905..."
258,경상남도,합천군,합천군,38400,48890,38400,48890,자치구,합천군,"POLYGON ((128.09861 35.83067, 128.09899 35.830..."
259,제주특별자치도,제주시,제주시,39010,50110,39010,50110,자치구,제주시,"MULTIPOLYGON (((126.15092 33.30877, 126.15090 ..."


In [31]:
sgg_gdf_process = sgg_gdf.copy()
sgg_gdf_process['geometry'] = sgg_gdf_process['geometry'].buffer(0.001) # 약간 크게 만들어서, 경계를 깔끔하게 병합하기
sgg_gdf_process = sgg_gdf_process.dissolve(by=['자치구_행정구역코드', '자치구_법정동코드']).reset_index()
sgg_gdf_process['geometry'] = sgg_gdf_process['geometry'].buffer(-0.001)
sgg_gdf_process

C:\Users\rkddn\AppData\Local\Temp/ipykernel_35484/3746526824.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  sgg_gdf_process['geometry'] = sgg_gdf_process['geometry'].buffer(0.001) # 약간 크게 만들어서, 경계를 깔끔하게 병합하기
C:\Users\rkddn\AppData\Local\Temp/ipykernel_35484/3746526824.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  sgg_gdf_process['geometry'] = sgg_gdf_process['geometry'].buffer(-0.001)


,자치구_행정구역코드,자치구_법정동코드,geometry,시도,시군구,행정구역명,시군구_행정구역코드,시군구_법정동코드,비자치구,자치구명
0,11010,11110,"POLYGON ((126.94890 37.62325, 126.94890 37.623...",서울특별시,종로구,종로구,11010,11110,자치구,종로구
1,11020,11140,"POLYGON ((126.96155 37.55860, 126.96155 37.558...",서울특별시,중구,중구,11020,11140,자치구,중구
2,11030,11170,"POLYGON ((126.94459 37.53378, 126.94460 37.533...",서울특별시,용산구,용산구,11030,11170,자치구,용산구
3,11040,11200,"POLYGON ((127.00815 37.54506, 127.00817 37.545...",서울특별시,성동구,성동구,11040,11200,자치구,성동구
4,11050,11215,"POLYGON ((127.05622 37.52832, 127.05623 37.528...",서울특별시,광진구,광진구,11050,11215,자치구,광진구
...,...,...,...,...,...,...,...,...,...,...
224,38380,48870,"POLYGON ((127.58486 35.55391, 127.58488 35.553...",경상남도,함양군,함양군,38380,48870,자치구,함양군
225,38390,48880,"POLYGON ((127.67944 35.76850, 127.67945 35.768...",경상남도,거창군,거창군,38390,48880,자치구,거창군
226,38400,48890,"POLYGON ((127.95323 35.53403, 127.95323 35.534...",경상남도,합천군,합천군,38400,48890,자치구,합천군
227,39010,50110,"MULTIPOLYGON (((126.94125 33.50648, 126.94125 ...",제주특별자치도,제주시,제주시,39010,50110,자치구,제주시


In [32]:
color_map = gpd.GeoDataFrame(sgg_gdf_process['시군구_행정구역코드'].unique(),columns=['시군구_행정구역코드'] )

for x in color_map.index:
    color = np.random.randint(0, 256, size=3)
    color = [str(hex(i))[2:] for i in color]
    color = '#'+''.join(color).upper()
    color_map.at[x, 'color'] = color
    
sgg_gdf_process_w_c = pd.merge(sgg_gdf_process, color_map, on='시군구_행정구역코드', how='inner')
sgg_gdf_process_w_c

,자치구_행정구역코드,자치구_법정동코드,geometry,시도,시군구,행정구역명,시군구_행정구역코드,시군구_법정동코드,비자치구,자치구명,color
0,11010,11110,"POLYGON ((126.94890 37.62325, 126.94890 37.623...",서울특별시,종로구,종로구,11010,11110,자치구,종로구,#594AF3
1,11020,11140,"POLYGON ((126.96155 37.55860, 126.96155 37.558...",서울특별시,중구,중구,11020,11140,자치구,중구,#E27B57
2,11030,11170,"POLYGON ((126.94459 37.53378, 126.94460 37.533...",서울특별시,용산구,용산구,11030,11170,자치구,용산구,#60531A
3,11040,11200,"POLYGON ((127.00815 37.54506, 127.00817 37.545...",서울특별시,성동구,성동구,11040,11200,자치구,성동구,#CE2073
4,11050,11215,"POLYGON ((127.05622 37.52832, 127.05623 37.528...",서울특별시,광진구,광진구,11050,11215,자치구,광진구,#C661AC
...,...,...,...,...,...,...,...,...,...,...,...
224,38380,48870,"POLYGON ((127.58486 35.55391, 127.58488 35.553...",경상남도,함양군,함양군,38380,48870,자치구,함양군,#263310
225,38390,48880,"POLYGON ((127.67944 35.76850, 127.67945 35.768...",경상남도,거창군,거창군,38390,48880,자치구,거창군,#97DA3
226,38400,48890,"POLYGON ((127.95323 35.53403, 127.95323 35.534...",경상남도,합천군,합천군,38400,48890,자치구,합천군,#5AAE7A
227,39010,50110,"MULTIPOLYGON (((126.94125 33.50648, 126.94125 ...",제주특별자치도,제주시,제주시,39010,50110,자치구,제주시,#9D285


In [35]:
sgg_gdf_process = sgg_gdf_process[['자치구_행정구역코드','자치구_법정동코드','시도','시군구','geometry']]
sgg_gdf_process = sgg_gdf_process.rename(columns={'자치구_행정구역코드':'SGG_AD_CD', '자치구_법정동코드':'SGG_LE_CD', '시도':'SIDO', '시군구':'SGG'})
sgg_gdf_process

,SGG_AD_CD,SGG_LE_CD,SIDO,SGG,geometry
0,11010,11110,서울특별시,종로구,"POLYGON ((126.94890 37.62325, 126.94890 37.623..."
1,11020,11140,서울특별시,중구,"POLYGON ((126.96155 37.55860, 126.96155 37.558..."
2,11030,11170,서울특별시,용산구,"POLYGON ((126.94459 37.53378, 126.94460 37.533..."
3,11040,11200,서울특별시,성동구,"POLYGON ((127.00815 37.54506, 127.00817 37.545..."
4,11050,11215,서울특별시,광진구,"POLYGON ((127.05622 37.52832, 127.05623 37.528..."
...,...,...,...,...,...
224,38380,48870,경상남도,함양군,"POLYGON ((127.58486 35.55391, 127.58488 35.553..."
225,38390,48880,경상남도,거창군,"POLYGON ((127.67944 35.76850, 127.67945 35.768..."
226,38400,48890,경상남도,합천군,"POLYGON ((127.95323 35.53403, 127.95323 35.534..."
227,39010,50110,제주특별자치도,제주시,"MULTIPOLYGON (((126.94125 33.50648, 126.94125 ..."


In [36]:
sgg_gdf_process.to_file('../data/0/geojson/sgg.geojson', driver='GeoJSON')

In [37]:
sgg_gdf_process.to_file('../data/0/shp/sgg.shp', encoding='UTF-8')